In [ ]:
import openai
import pandas as pd
import csv
import json
import time
import requests

# Set up OpenAI with your API key
openai.api_key = ''  # Replace with your actual API key

# Load the dataset
df = pd.read_csv('reddit_data_processed.csv')

# Function to query your fine-tuned model
def query_model_with_retry(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = openai.Completion.create(
                model="ft:gpt-3.5-turbo-1106:personal::8PG4w9zg",
                prompt=prompt,
                max_tokens=100,
                timeout=180
            )
            return response.choices[0].text.strip()
        except requests.exceptions.Timeout:
            if attempt < max_retries - 1:
                time.sleep(5)  # wait for 5 seconds before retrying
                continue
            else:
                raise

# Define patterns indicative of certain types of misinformation
misinformation_patterns = {
    "False Claims about Side Effects": [
        "side effect", "adverse reaction", "harmful effect", "causes harm",
        "health risk", "unsafe", "dangerous", "negative impact"
    ],
    "Conspiracy Theories": [
        "conspiracy", "plandemic", "hoax", "manipulation", "government control",
        "hidden agenda", "secret plan", "big pharma conspiracy", "microchip"
    ],
    "Misconceptions about Vaccine Efficacy": [
        "ineffective", "doesn't work", "useless", "no benefit", "not effective",
        "waste of time", "pointless", "no protection", "doesn't help"
    ],
    "Misinformation on Vaccine Ingredients": [
        "toxic ingredients", "dangerous chemicals", "harmful substances",
        "contains mercury", "contains aluminum", "unsafe compounds"
    ],
    "Misinformation on Vaccine Necessity": [
        "not necessary", "unnecessary", "no need", "not needed", "overstated need"
    ],
    "Anti-Vaccination": [
        "anti-vax", "vaccine skeptic", "vaccine refusal", "refuse vaccine",
        "vaccine hesitant", "do not vaccinate", "against vaccination"
    ]
    # Add more categories and patterns as relevant to your analysis
}


# Parsing function to infer the misinformation category
def infer_misinformation_category(model_response):
    for category, patterns in misinformation_patterns.items():
        if any(pattern in model_response.lower() for pattern in patterns):
            return category
    return "No Misinformation" if "no misinformation" in model_response.lower() else "Uncategorized"

# Open CSV and JSON files for writing
csv_file = open('real_time_analysis_results.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.DictWriter(csv_file, fieldnames=["Keyword", "Title", "URL", "Comment", "Model_Response", "Misinformation_Category"])
csv_writer.writeheader()

json_file = open('real_time_analysis_results.json', 'w', encoding='utf-8')
json_file.write('[')
first_record = True

for index, row in df.iterrows():
    prompt = f"Analyze the following comment for potential vaccine misinformation:\n\n{row['Comment']}\n"
    model_response = query_model_with_retry(prompt)
    misinformation_category = infer_misinformation_category(model_response)

    # Construct the record
    record = {
        "Keyword": row['Keyword'],
        "Title": row['Title'],
        "URL": row['URL'],
        "Comment": row['Comment'],
        "Model_Response": model_response,
        "Misinformation_Category": misinformation_category
    }

    # Write to CSV
    csv_writer.writerow(record)

    # Write to JSON
    if not first_record:
        json_file.write(',\n')
    json.dump(record, json_file, indent=4)
    first_record = False

    # Optional: Add a delay to avoid hitting API rate limits
    time.sleep(1)

# Close JSON array and file
json_file.write('\n]')
json_file.close()

# Close CSV file
csv_file.close()

print("Real-time analysis completed and data saved.")